In [18]:
##Stacked GRU - 2layers with attention mechanism 



try:
    import tensorflow
except ImportError:
    # install qlib
    !pip install gast==0.2.2
    !pip install tensorflow==1.14.0
    
from tensorflow.contrib.rnn.python.ops.rnn_cell import AttentionCellWrapper
from tensorflow.contrib.rnn.python.ops.gru_ops import GRUBlockCellV2
from tensorflow.compat.v1.nn.rnn_cell import DropoutWrapper
    
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import tensorflow as tf
import talib 
import datetime
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
import time
import os
pd.set_option('display.max_rows', 500)
try:
    os.chdir('Models')
except:
    pass

print(os.getcwd())
batch_size = 100
in_length = 48
#504 是 (240/30) * (feature 63) 
in_width = 504
num_units = 128
num_dims = 2
outclasses = 3
#learning_rate = 3e-4
learning_rate = 0.001
max_grad_norm = 5

start = '2016-06-01'
end = '2021-01-01'


cells_f = []
cells_b = []
n_layers = 3

tf.compat.v1.reset_default_graph()



keep_prob = tf.placeholder(shape=[], dtype=tf.float32)
keep_in_prob = tf.placeholder(shape=[], dtype=tf.float32)

batch_sizee = tf.placeholder(tf.int64)
MAX_COMP = 11

attention_layer_fw1= GRUBlockCellV2(num_units=n_in)

attention_layer_fw1 = AttentionCellWrapper(attention_layer_fw1,attn_length=2)

attention_layer_fw1 = tf.contrib.rnn.DropoutWrapper(attention_layer_fw1,output_keep_prob=keep_prob,input_keep_prob=keep_in_prob)

cells_f.append(attention_layer_fw1)


###

attention_layer_fw2= GRUBlockCellV2(num_units=n_in)

attention_layer_fw2 = AttentionCellWrapper(attention_layer_fw2,attn_length=2)

attention_layer_fw2 = tf.contrib.rnn.DropoutWrapper(attention_layer_fw2,output_keep_prob=keep_prob,input_keep_prob=keep_in_prob)

cells_f.append(attention_layer_fw2)


###



cell_f = tf.contrib.rnn.MultiRNNCell(cells_f, state_is_tuple=True)
    




input_data = tf.placeholder(shape=[None, in_length, 8,col_after_drop], dtype=tf.float32)

X_input = tf.transpose(a=input_data, perm=[1,0,2,3])


targets = tf.placeholder(shape=[None,outclasses], dtype=tf.float32)


X_input_2  = tf.reshape(X_input,[in_length,-1,8*col_after_drop])


X_input_3 = tf.layers.dense(X_input_2,units=in_width,activation=tf.nn.tanh, name='in_fc',
    kernel_initializer=tf.truncated_normal_initializer())




outputs,new_state = tf.nn.dynamic_rnn(cell=cell_f, \
                          inputs=X_input_3, \
                          initial_state=None, \
                          time_major=True,\
                             dtype=tf.float32)



Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [20]:
output_sequences = outputs
loss = 0
adv_loss = 0
l2_norm = 0

adv_train= 1
hinge = 1
att=1

In [21]:
with tf.variable_scope('lstm2') as scope:
    ##myloss:
    
    
    outputs = tf.transpose(output_sequences,[1,0,2])
    
    
    av_W = tf.get_variable(
      name='att_W', dtype=tf.float32,
      shape=[n_in*1,n_in*1],
      initializer=tf.glorot_uniform_initializer()
      )
    av_b = tf.get_variable(
          name='att_h', dtype=tf.float32,
          shape=[n_in*1],
          initializer=tf.zeros_initializer())
        
    av_u = tf.get_variable(
          name='att_u', dtype=tf.float32,
          shape=[n_in*1],
          initializer=tf.glorot_uniform_initializer()
      )

    a_laten = tf.tanh(
          tf.tensordot(outputs,av_W,
                        axes=1) + av_b)
    a_scores = tf.tensordot(a_laten, av_u,
                                    axes=1,
                                    name='scores')
    a_alphas = tf.nn.softmax(a_scores, name='alphas')

    a_con = tf.reduce_sum(
          outputs * tf.expand_dims(a_alphas, -1), 1)
    
    fea_con = tf.concat(
      [outputs[:, -1, :], a_con],
      axis=1)
    
 


In [22]:
with tf.variable_scope('pre_fcc2'):
    
    
    fc_W = tf.get_variable(
        'weights', dtype=tf.float32,
        shape=[n_in*1*2, outclasses],
        initializer=tf.glorot_uniform_initializer()
    )
    fc_b = tf.get_variable(
        'biases', dtype=tf.float32,
        shape=[outclasses],
        initializer=tf.zeros_initializer()
    )

########my algo is basically upto this part, pred of this code equals final_output of my code##
   
    final_output = tf.nn.bias_add(
            tf.matmul(fea_con, fc_W), fc_b
        )
    

    
    
   

In [23]:

cost = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=final_output, labels=targets))

loss =cost


##ADV Loss is implemented below: ##

adv_loss = loss * 0
#tf.compat.v1.reset_default_graph()
#outputs = tf.reverse(tensor=outputs, axis=[0])[0,:,:]
delta_adv= tf.gradients(loss, [fea_con])[0]
#tf.stop_gradient(delta_adv)
delta_adv = tf.stop_gradient(delta_adv)
delta_adv = tf.nn.l2_normalize(delta_adv, axis=1)
adv_pv_var = fea_con + \
                        0.6 * delta_adv
scope.reuse_variables()

with tf.variable_scope('pre_fcc2',reuse=True):
    fc_W = tf.get_variable(
        'weights', dtype=tf.float32,
        shape=[n_in*1*2, outclasses],
        initializer=tf.truncated_normal_initializer()

    )
    fc_b = tf.get_variable(
        'biases', dtype=tf.float32,
        shape=[outclasses],
        initializer=tf.zeros_initializer()
    )



    adv_pred = tf.nn.bias_add(
            tf.matmul(adv_pv_var, fc_W), fc_b
        )
    


adv_loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=adv_pred, labels=targets))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [24]:

correct_pred = tf.equal(tf.argmax(final_output,1), tf.argmax(targets,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

##Using the entire dataset - but this could be divided into training , validation and testing sets if required

test_X = trainX
test_Y = trainY


loss_list = []
acc_list = []
acc_pred_list = []
adv_loss_list = []
saver = tf.train.Saver()


In [29]:
#with tf.variable_scope('lstm2') as scope:
global_step = tf.Variable(0, trainable=False, name="global_step")
lr = tf.train.exponential_decay(
                learning_rate = learning_rate,
                decay_steps=10,
                global_step = global_step,
                decay_rate = 0.85,
                staircase=True
            )

_trainable_variables = tf.trainable_variables()
l2_coefficient = 97e-3
_l2_regularizer = tf.add_n([tf.nn.l2_loss(v) for v in _trainable_variables])
_l2_regularizer = _l2_regularizer * l2_coefficient / len(_trainable_variables)



obj_func = cost +lr * adv_loss +_l2_regularizer 

# cost1 = tf.reduce_sum(tf.nn.weighted_cross_entropy_with_logits(logits=final_output, labels=targets,pos_weight=0.8))
# cost = cost1+ _l2_regularizer

params = tf.trainable_variables()
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
gvs = optimizer.compute_gradients(obj_func, params)
ysss = []
for i, (grid,var) in enumerate(gvs):
    if grid != None:
        grid = tf.clip_by_value(grid,-5.,5.)
        gvs[i] = (grid,var)
optimizer = optimizer.apply_gradients(gvs,global_step = global_step)

model_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

is_training = tf.get_variable('is_training', shape=(), dtype=tf.bool,
                              initializer=tf.constant_initializer(True, dtype=tf.bool))


ema = tf.train.ExponentialMovingAverage(decay=0.9988)

with tf.control_dependencies([optimizer]):
    
    train_op = ema.apply(model_vars)
    
    
# Make backup variables
# Make backup variables
# Make backup variables

with tf.variable_scope('BackupVariables'):
    backup_vars = [tf.get_variable(var.op.name, dtype=var.value().dtype, trainable=False,
                                   initializer=var.initialized_value())
                   for var in model_vars]
    

def ema_to_weights():
    return tf.group(*(tf.assign(var, ema.average(var).read_value())
                     for var in model_vars))
def save_weight_backups():
    return tf.group(*(tf.assign(bck, var.read_value())
                     for var, bck in zip(model_vars, backup_vars)))
def restore_weight_backups():
    return tf.group(*(tf.assign(var, bck.read_value())
                     for var, bck in zip(model_vars, backup_vars)))

def to_training():
    with tf.control_dependencies([tf.assign(is_training, True)]):
        return restore_weight_backups()

def to_testing():
    with tf.control_dependencies([tf.assign(is_training, False)]):
        with tf.control_dependencies([save_weight_backups()]):
            return ema_to_weights()
        
    
switch_to_train_mode_op = tf.cond(is_training, lambda: tf.group(), to_training)
switch_to_test_mode_op = tf.cond(is_training, to_testing, lambda: tf.group())


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


In [32]:
#####Training Begins Here######
#####Training Begins Here######
#####Training Begins Here######
#####Training Begins Here######
#####Training Begins Here######

save_flag = False
program_starts = time.time()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saved_path = saver.restore(sess,stk1[:6]+'tmt.ckpt')
    sess.run(switch_to_train_mode_op)
    for epoch in np.arange(500):
        print(epoch)
        # for i in np.arange(int(len(trainX)/batch_size)):
        loop_starts = time.time()
        batch_x = trainX
        batch_y = trainY
        sess.run(train_op, feed_dict={input_data: batch_x, targets: batch_y,keep_prob:0.95,keep_in_prob: 0.5,batch_sizee:np.int64(batch_x.shape[0])})
        print('global_step: %s' % tf.compat.v1.train.global_step(sess,global_step))

        if epoch % 5 == 0 or epoch==284 or loss_list[-1] < 50:
            acc, adv_loss_2,loss_2= sess.run([accuracy, adv_loss,loss], feed_dict={input_data: batch_x, targets: batch_y,keep_prob: 1.0,keep_in_prob: 1.0,batch_sizee:np.int64(batch_x.shape[0])})
            print(acc,"LOSS:  ",loss_2, "ADV_LOSS:  ",adv_loss_2)
            saved_path = saver.save(sess,stk1[:6]+'tmt.ckpt')

            if (loss_2 < 40 and loss_list[-1] - loss_2 < 4 and acc >= 0.99) or epoch == 184:
                saved_path = saver.save(sess,stk1[:6]+'tmt.ckpt')
                
                while not save_flag:
            
                    try:

                        sess.run(switch_to_test_mode_op)

                        acc, adv_loss_2,loss_2= sess.run([accuracy, adv_loss,loss], feed_dict={input_data: batch_x, targets: batch_y,keep_prob: 1.0,keep_in_prob: 1.0,batch_sizee:np.int64(batch_x.shape[0])})
                        print(acc,"LOSS:  ",loss_2, "ADV_LOSS:  ",adv_loss_2)
                        saved_path = saver.save(sess,stk1[:7]+'tmt.ckpt')
                        #sess.run(switch_to_train_mode_op)
                        save_flag = True
                        print("sucessfully saved")
                        
                    except:
                        print('failed to save')
                        pass


                break
                
            loss_list.append(loss_2)
            acc_list.append(acc)
        pred = sess.run(accuracy, feed_dict={input_data: test_X, targets: test_Y,keep_prob: 1.0,keep_in_prob: 1.0,batch_sizee:np.int64(test_X.shape[0])})
        acc_pred_list.append(pred)
        end_loop = time.time()
        
        print("It has been {0} seconds since the loop started".format(end_loop - loop_starts))
        saved_path = saver.save(sess,stk1[:6]+'tmt.ckpt')
        
now = time.time()
print("It has been {0} seconds since the ENTIRE THING started".format(now - program_starts))

0
global_step: 1
0.6942308 LOSS:   951.8249 ADV_LOSS:   1708.7864
It has been 93.01626229286194 seconds since the loop started
1
global_step: 2
It has been 70.72398352622986 seconds since the loop started
2
global_step: 3
It has been 72.17823910713196 seconds since the loop started
3
global_step: 4
It has been 72.42990779876709 seconds since the loop started
4
global_step: 5
It has been 72.55018496513367 seconds since the loop started
5
global_step: 6
0.69711536 LOSS:   901.57385 ADV_LOSS:   1636.4192
It has been 92.03213858604431 seconds since the loop started
6
global_step: 7
It has been 74.30415511131287 seconds since the loop started
7
global_step: 8
It has been 74.77837228775024 seconds since the loop started
8
global_step: 9
It has been 74.66608810424805 seconds since the loop started
9
global_step: 10
It has been 73.97279453277588 seconds since the loop started
10
global_step: 11
0.70096153 LOSS:   811.63184 ADV_LOSS:   1473.1483
It has been 92.99013566970825 seconds since the l

It has been 78.27773976325989 seconds since the loop started
92
global_step: 93
It has been 76.27825164794922 seconds since the loop started
93
global_step: 94
It has been 78.89975333213806 seconds since the loop started
94
global_step: 95
It has been 77.74976563453674 seconds since the loop started
95
global_step: 96
0.9221154 LOSS:   200.04599 ADV_LOSS:   443.9931
It has been 98.88061261177063 seconds since the loop started
96
global_step: 97
It has been 75.36745858192444 seconds since the loop started
97
global_step: 98
It has been 74.74681973457336 seconds since the loop started
98
global_step: 99
It has been 77.86624503135681 seconds since the loop started
99
global_step: 100
It has been 77.89281845092773 seconds since the loop started
100
global_step: 101
0.91923076 LOSS:   198.29224 ADV_LOSS:   442.78696
It has been 97.54989194869995 seconds since the loop started
101
global_step: 102
It has been 77.45481896400452 seconds since the loop started
102
global_step: 103
It has been 7

181
global_step: 182
It has been 82.74758100509644 seconds since the loop started
182
global_step: 183
It has been 83.01431179046631 seconds since the loop started
183
global_step: 184
It has been 80.86888337135315 seconds since the loop started
184
global_step: 185
It has been 81.14150214195251 seconds since the loop started
185
global_step: 186
0.9548077 LOSS:   145.06113 ADV_LOSS:   333.35913
It has been 101.65698838233948 seconds since the loop started
186
global_step: 187
It has been 81.02862238883972 seconds since the loop started
187
global_step: 188
It has been 81.26349425315857 seconds since the loop started
188
global_step: 189
It has been 80.34475922584534 seconds since the loop started
189
global_step: 190
It has been 80.46553087234497 seconds since the loop started
190
global_step: 191
0.95 LOSS:   142.2545 ADV_LOSS:   330.7081
It has been 103.1449704170227 seconds since the loop started
191
global_step: 192
It has been 74.52345895767212 seconds since the loop started
192


KeyboardInterrupt: 

In [33]:
class Model(object):

  def __init__(self, model_path,period):
    # Note, if you don't want to leak this, you'll want to turn Model into
    # a context manager. In practice, you probably don't have to worry
    # about it.
    self.model_path = model_path
    self.session = tf.Session()
    self.period = period
    #self.session.run(tf.global_variables_initializer())
   
    saver.restore(self.session,self.model_path)

    #self.softmax_tensor = self.session.graph.get_tensor_by_name('final_ops/softmax:0')
    self.clas = tf.nn.softmax(final_output)
    #self.clas = final_output
    #tg = sess.run([clas], feed_dict={input_data:trainX})
  def predict(self, images):
    predictions = self.session.run(self.clas, {input_data: images,keep_prob: 1.0, keep_in_prob: 1.0,batch_sizee:np.int64(self.period)})
    # TODO: convert to human-friendly labels
    return predictions

In [34]:
model = Model(stk1[:6]+'tmt.ckpt',137)
print('模型： \t',stk1[:6]+'tmt.ckpt')

Instructions for updating:
Use standard file APIs to check for files with this prefix.


模型： 	 600881tmt.ckpt
